In [1]:
LOAD_WORD  = 0x01
STORE_WORD = 0x02
ADD        = 0x03
SUB        = 0x04
ADD_ONE    = 0x05 # second parameter unused
HALT       = 0xff

In [2]:
def from_twos(value, bits=16):
    assert 0 <= value < 2 ** bits
    if value < 2 ** bits / 2:
        return value
    else:
        return value - 2 ** bits

In [3]:
def to_twos(value, bits=16):
    assert -2 ** bits / 2 <= value < 2 ** bits / 2
    if value >= 0:
        return value
    else:
        return 2 ** bits + value

In [4]:
bits = 3
for n in range(2**bits):
    print(n, from_twos(n, bits), to_twos(from_twos(n, bits), bits))

0 0 0
1 1 1
2 2 2
3 3 3
4 -4 4
5 -3 5
6 -2 6
7 -1 7


In [5]:
memory = bytearray(
    [
        0x01, 0x01, 0x10, # instructions
        0x01, 0x02, 0x12,
        0x04, 0x01, 0x02,
        0x02, 0x01, 0x0e,
        0xff,
        0x00,             # reserved empty byte
        0x00, 0x00,       # output
        0xf4, 0xff,       # input 1
        0x9c, 0xff        # input 2
    ]
)

assert len(memory) == 20

In [6]:
def run(memory):
    registers = [0, 0, 0]
    
    print('input 1:', from_twos(memory[0x10] + 256 * memory[0x11]))
    print('input 2:', from_twos(memory[0x12] + 256 * memory[0x13]))
    
    while True:
        pc = registers[0]
        instruction = memory[pc]
        if instruction == HALT:
            print('HALT')
            break
        elif instruction == LOAD_WORD:
            print('LOAD')
            r_idx = memory[pc + 1]
            in_idx = memory[pc + 2]
            registers[r_idx] = memory[in_idx] + 256 * memory[in_idx + 1]
        elif instruction == STORE_WORD:
            print('STORE')
            r_idx = memory[pc + 1]
            in_idx = memory[pc + 2]
            value = registers[r_idx]
            memory[in_idx] = value % 256
            memory[in_idx + 1] = value // 256
        elif instruction == ADD:
            print('ADD')
            left = memory[pc + 1]
            right = memory[pc + 2]
            registers[left] = to_twos(from_twos(registers[left]) + from_twos(registers[right]))
        elif instruction == SUB:
            print('SUB')
            left = memory[pc + 1]
            right = memory[pc + 2]
            registers[left] = to_twos(from_twos(registers[left]) - from_twos(registers[right]))
        elif instruction == ADD_ONE:
            print('ADD ONE')
            left = memory[pc + 1]
            registers[left] = to_twos(from_twos(registers[left]) + 1)
        else:
            print('Should not get here.')
            return

        registers[0] += 3

        print(memory, registers)


    print('output', from_twos(memory[0x0e] + 256 * memory[0x0f]))

In [7]:
run(memory)

input 1: -12
input 2: -100
LOAD
bytearray(b'\x01\x01\x10\x01\x02\x12\x04\x01\x02\x02\x01\x0e\xff\x00\x00\x00\xf4\xff\x9c\xff') [3, 65524, 0]
LOAD
bytearray(b'\x01\x01\x10\x01\x02\x12\x04\x01\x02\x02\x01\x0e\xff\x00\x00\x00\xf4\xff\x9c\xff') [6, 65524, 65436]
SUB
bytearray(b'\x01\x01\x10\x01\x02\x12\x04\x01\x02\x02\x01\x0e\xff\x00\x00\x00\xf4\xff\x9c\xff') [9, 88, 65436]
STORE
bytearray(b'\x01\x01\x10\x01\x02\x12\x04\x01\x02\x02\x01\x0e\xff\x00X\x00\xf4\xff\x9c\xff') [12, 88, 65436]
HALT
output 88
